In [1]:
!pip install transformers

You should consider upgrading via the 'c:\users\aniru\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [4]:
!pip install sentencepiece

You should consider upgrading via the 'c:\users\aniru\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


# Setup summarisation model

In [19]:
# model_name = "human-centered-summarization/financial-summarization-pegasus"
# tokenizer = PegasusTokenizer.from_pretrained(model_name)
# model = PegasusForConditionalGeneration.from_pretrained(model_name)

In [6]:
monitored_tickers = ['BTC']

In [7]:
def search_for_stock_news_urls(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&tbm=nws".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs

In [8]:
raw_urls = {ticker:search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls

{'BTC': ['/?sa=X&ved=0ahUKEwiWqKv4v-nwAhUjHbkGHSpzD8wQOwgC',
  '/?output=search&ie=UTF-8&tbm=nws&sa=X&ved=0ahUKEwiWqKv4v-nwAhUjHbkGHSpzD8wQPAgE',
  '/search?q=yahoo+finance+BTC&tbm=nws&ie=UTF-8&gbv=1&sei=xF-vYJa2GaO65OUPqua94Aw',
  '/search?q=yahoo+finance+BTC&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwiWqKv4v-nwAhUjHbkGHSpzD8wQ_AUIBygA',
  '/search?q=yahoo+finance+BTC&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwiWqKv4v-nwAhUjHbkGHSpzD8wQ_AUICSgC',
  '/search?q=yahoo+finance+BTC&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwiWqKv4v-nwAhUjHbkGHSpzD8wQ_AUICigD',
  '/search?q=yahoo+finance+BTC&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiWqKv4v-nwAhUjHbkGHSpzD8wQ_AUICygE',
  'https://maps.google.com/maps?q=yahoo+finance+BTC&um=1&ie=UTF-8&sa=X&ved=0ahUKEwiWqKv4v-nwAhUjHbkGHSpzD8wQ_AUIDCgF',
  '/search?q=yahoo+finance+BTC&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwiWqKv4v-nwAhUjHbkGHSpzD8wQ_AUIDSgG',
  '/advanced_search',
  '/search?q=yahoo+finance+BTC&ie=UTF-8&tbm=nws&source=lnt&tbs=qdr:h&s

In [10]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [11]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls: 
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [12]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_tickers}
cleaned_urls

{'BTC': ['https://finance.yahoo.com/news/why-china-crackdown-may-bitcoin-015850545.html',
  'https://finance.yahoo.com/news/forget-bitcoin-ethereum-10-best-161610562.html',
  'https://finance.yahoo.com/video/bitcoin-could-touch-300-000-191436168.html',
  'https://finance.yahoo.com/video/bitcoin-wild-price-action-continues-132126621.html',
  'https://finance.yahoo.com/video/rising-inflation-could-mean-bitcoin-211557985.html',
  'https://finance.yahoo.com/news/course-china-anti-bitcoin-look-230807400.html',
  'https://finance.yahoo.com/news/bitcoin-rises-above-40k-mining-083312103.html',
  'https://finance.yahoo.com/news/bitcoin-is-officially-a-new-asset-class-goldman-sachs-103540636.html',
  'https://finance.yahoo.com/news/fidelity-bitcoin-fund-attracts-102m-200345900.html',
  'https://finance.yahoo.com/video/bitcoin-cryptocurrencies-rebound-volatile-week-135012937.html']}

In [13]:
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs: 
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [14]:
articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'BTC': ['China’s nationwide crackdown on crypto mining could create a competitive environment\xa0 in which only the biggest miners can survive, said a co-founder and managing partner of Waterdrip Capital, a major investor in the Chinese crypto mining industry. “Finding suitable sites outside the mining hubs requires a lot of resources such as capital and network,” Yusan Zheng said during a Chinese-language Clubhouse discussion hosted by CoinDesk. “Only the most experienced and deep-pocketed miners will be able to carry out such a plan.”\xa0 Local authorities in major Chinese mining hubs have started banishing crypto mining businesses since last Friday’s crackdown notice from the State Council. With little idle capacity in overseas hosting sites, some miners plan to go underground and continue to operate in other parts of the country, which might be only feasible for some of the biggest miners. Related: DeFi &#8216;Raises Challenges&#8217; for Investors, Regulators, SEC&#8217;s Gensler

In [15]:
def summarize(articles):
    summaries = []
    for article in articles:
        input_ids = tokenizer.encode(article, return_tensors='pt')
        output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [16]:
summaries = {ticker:summarize(articles[ticker]) for ticker in monitored_tickers}
summaries

AttributeError: 'NoneType' object has no attribute 'encode'